In [1]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行

# load data

In [50]:
df_sales = pd.read_csv('data_origin/train/train_sales_data.csv')
df_search = pd.read_csv('data_origin/train/train_search_data.csv')
df_user_reply = pd.read_csv('data_origin/train/train_user_reply_data.csv')

df = pd.merge(df_sales, df_search, on='province adcode model regYear regMonth'.split())

df = pd.merge(df, df_user_reply, on='model regYear regMonth'.split())

In [40]:
df.head()

,province,adcode,model,bodyType,regYear,regMonth,salesVolume,popularity,carCommentVolum,newsReplyVolum
0,上海,310000,3c974920a76ac9c1,SUV,2016,1,292,1479,11,106
1,云南,530000,3c974920a76ac9c1,SUV,2016,1,466,1594,11,106
2,内蒙古,150000,3c974920a76ac9c1,SUV,2016,1,257,1479,11,106
3,北京,110000,3c974920a76ac9c1,SUV,2016,1,408,2370,11,106
4,四川,510000,3c974920a76ac9c1,SUV,2016,1,610,3562,11,106


In [28]:
df.tail()

,province,adcode,model,bodyType,regYear,regMonth,salesVolume,popularity,carCommentVolum,newsReplyVolum
31675,福建,350000,a9a43d1a7ecbe75d,SUV,2017,12,112,772,339,377
31676,辽宁,210000,a9a43d1a7ecbe75d,SUV,2017,12,104,1049,339,377
31677,重庆,500000,a9a43d1a7ecbe75d,SUV,2017,12,125,654,339,377
31678,陕西,610000,a9a43d1a7ecbe75d,SUV,2017,12,265,1516,339,377
31679,黑龙江,230000,a9a43d1a7ecbe75d,SUV,2017,12,58,736,339,377


## test数据

In [51]:
df_test = pd.read_csv('data_origin/evaluation_public.csv')
# 添加bodyType
dft = df[['bodyType', 'model']].drop_duplicates()
df_test = pd.merge(df_test, dft, on='model', how='left')
del dft

df_test = df_test.rename(columns={'forecastVolum': 'salesVolume'})

df = pd.concat([df, df_test], sort=False, ignore_index=True)

In [8]:
# # add 'popularity', 'carCommentVolum', 'newsReplyVolum'
# def t(s):
#     return int(s.mean())
# dfg = df.groupby('model regMonth'.split())[['popularity', 'carCommentVolum', 'newsReplyVolum']].agg(t)

# df_test = pd.merge(df_test, dfg, left_on='model regMonth'.split(), right_index=True, how='left')

In [52]:
del df['adcode']

In [53]:
cols_cat = ['province', 'model', 'bodyType', 'regYear', 'regMonth']
cols_num = ['popularity', 'carCommentVolum', 'newsReplyVolum']
label = 'salesVolume'

# time_id

In [54]:
df['t'] = 0
df.loc[df.regYear==2017, 't'] = 12
df.loc[df.regYear==2018, 't'] = 24
df['time_id'] = df.regMonth + df.t
df['time_id'] = df.time_id.map(int)
del df['t']

# 强相关特征

## 历史月数据 salesVolume | popularity

In [15]:
# def ft(col, df_, n):
#     df.loc[:, f'{col}_last_{n}'] = df.groupby('province model'.split())[col].shift(n)

In [16]:
# colst1 = ['salesVolume']
# colst2 = ['salesVolume', 'popularity']
# for i in range(6):
#     df.loc[:, f'salesVolume_last_{i+1}'] = df.groupby('province model'.split())['salesVolume'].shift(i+1)
#     df.loc[:, f'popularity_last_{i+1}'] = df.groupby('province model'.split())['popularity'].shift(i+1)
#     colst1.extend([f'salesVolume_last_{i+1}'])
#     colst2.extend([f'popularity_last_{i+1}'])

In [55]:
# colst1 = ['salesVolume']
# colst2 = ['salesVolume', 'popularity']
for i in range(6):
    df.loc[:, f'salesVolume_last_{i+1}'] = df.groupby('province model'.split())['salesVolume'].shift(i+1)
#     colst1.extend([f'salesVolume_last_{i+1}'])
#     df.loc[:, f'popularity_last_{i+1}'] = df.groupby('province model'.split())['popularity'].shift(i+1)
#     colst2.extend([f'popularity_last_{i+1}'])

In [44]:
df[colst1].corr()

,salesVolume,salesVolume_last_1,salesVolume_last_2,salesVolume_last_3,salesVolume_last_4,salesVolume_last_5,salesVolume_last_6,salesVolume_last_7,salesVolume_last_8,salesVolume_last_9,salesVolume_last_10,salesVolume_last_11,salesVolume_last_12
salesVolume,1.000000,0.906629,0.869059,0.857891,0.843783,0.811136,0.795037,0.797457,0.799980,0.791562,0.786466,0.871551,0.924145
salesVolume_last_1,0.906629,1.000000,0.907406,0.869899,0.857840,0.843405,0.813241,0.799413,0.800579,0.808090,0.801559,0.826100,0.896174
salesVolume_last_2,0.869059,0.907406,1.000000,0.903304,0.863638,0.851599,0.838451,0.808118,0.791932,0.795259,0.803183,0.814819,0.828754
salesVolume_last_3,0.857891,0.869899,0.903304,1.000000,0.898933,0.857736,0.846812,0.834212,0.800972,0.785714,0.788603,0.812264,0.815255
salesVolume_last_4,0.843783,0.857840,0.863638,0.898933,1.000000,0.893562,0.852345,0.841959,0.828121,0.794896,0.778033,0.799778,0.814890
salesVolume_last_5,0.811136,0.843405,0.851599,0.857736,0.893562,1.000000,0.889259,0.846941,0.835463,0.822917,0.787511,0.787659,0.801774
salesVolume_last_6,0.795037,0.813241,0.838451,0.846812,0.852345,0.889259,1.000000,0.885213,0.840501,0.829728,0.816906,0.791948,0.786247
salesVolume_last_7,0.797457,0.799413,0.808118,0.834212,0.841959,0.846941,0.885213,1.000000,0.880839,0.834175,0.823003,0.820900,0.790368
salesVolume_last_8,0.799980,0.800579,0.791932,0.800972,0.828121,0.835463,0.840501,0.880839,1.000000,0.876364,0.827327,0.826610,0.819531
salesVolume_last_9,0.791562,0.808090,0.795259,0.785714,0.794896,0.822917,0.829728,0.834175,0.876364,1.000000,0.871424,0.829137,0.825715


## 前n个月的累计值 salesVolume | popularity

In [56]:
# colst1 = ['salesVolume', 'popularity']
# colst2 = ['salesVolume', 'popularity']
for i in range(2, 6):
    df.loc[:, f'salesVolume_sum_{i}'] = df.groupby('province model'.split())['salesVolume'].rolling(i).sum().reset_index('province model'.split(), drop=True)
    df.loc[:, f'salesVolume_sum_{i}'] = df.groupby('province model'.split())[f'salesVolume_sum_{i}'].shift(1)
    
#     df.loc[:, f'popularity_sum_{i}'] = df.groupby('province model'.split())['popularity'].rolling(i).sum().reset_index('province model'.split(), drop=True)
#     df.loc[:, f'popularity_sum_{i}'] = df.groupby('province model'.split())[f'popularity_sum_{i}'].shift(1)
    
#     colst1.extend([f'salesVolume_sum_{i}'])
#     colst2.extend([f'popularity_sum_{i}'])

In [92]:
df[colst1].corr()

,salesVolume,popularity,salesVolume_sum_2,salesVolume_sum_3,salesVolume_sum_4,salesVolume_sum_5
salesVolume,1.000000,0.474152,0.917529,0.917912,0.916105,0.908731
popularity,0.474152,1.000000,0.485585,0.490227,0.492200,0.491176
salesVolume_sum_2,0.917529,0.485585,1.000000,0.988983,0.980060,0.972894
salesVolume_sum_3,0.917912,0.490227,0.988983,1.000000,0.993642,0.987354
salesVolume_sum_4,0.916105,0.492200,0.980060,0.993642,1.000000,0.995750
salesVolume_sum_5,0.908731,0.491176,0.972894,0.987354,0.995750,1.000000


In [93]:
df[colst2].corr()

,salesVolume,popularity,popularity_sum_2,popularity_sum_3,popularity_sum_4,popularity_sum_5
salesVolume,1.000000,0.474152,0.486602,0.492305,0.493733,0.493911
popularity,0.474152,1.000000,0.827553,0.812140,0.802557,0.794597
popularity_sum_2,0.486602,0.827553,1.000000,0.981814,0.957805,0.939753
popularity_sum_3,0.492305,0.812140,0.981814,1.000000,0.988921,0.972504
popularity_sum_4,0.493733,0.802557,0.957805,0.988921,1.000000,0.992499
popularity_sum_5,0.493911,0.794597,0.939753,0.972504,0.992499,1.000000


# 趋势特征

## 与前n个月差值

In [57]:
# colst1 = ['salesVolume']
# colst2 = ['salesVolume']
# colst3 = ['salesVolume']
# colst4 = ['salesVolume']
for i in range(1, 6):
    df.loc[:, f'salesVolume_diff_{i}_{i+1}'] = df[f'salesVolume_last_{i}'] - df[f'salesVolume_last_{i+1}']
#     df.loc[:, f'salesVolume_diff_{i}_{i+1}'] = df.groupby('province model'.split())[f'salesVolume_diff_{i}_{i+1}'].shift(1)
    df.loc[:, f'salesVolume_diff_{i}_{i+1}_ratio'] = df.loc[:, f'salesVolume_diff_{i}_{i+1}']/(df[f'salesVolume_last_{i+1}']+1)
    
#     df.loc[:, f'popularity_diff_{i+1}'] = df.salesVolume - df[f'popularity_last_{i+1}']
#     df.loc[:, f'popularity_diff_{i+1}'] = df.groupby('province model'.split())[f'popularity_diff_{i+1}'].shift(1)
#     df.loc[:, f'popularity_diff_{i+1}_ratio'] = df.loc[:, f'popularity_diff_{i+1}']/(df.loc[:, f'popularity']+1)
    
#     colst1.append(f'salesVolume_diff_{i+1}')
#     colst2.append(f'salesVolume_diff_{i+1}_ratio')
#     colst3.append(f'popularity_diff_{i+1}')
#     colst4.append(f'popularity_diff_{i+1}_ratio')

In [75]:
df[colst1].corr()

,salesVolume,salesVolume_diff_1,salesVolume_diff_2,salesVolume_diff_3,salesVolume_diff_4,salesVolume_diff_5,salesVolume_diff_6
salesVolume,1.000000,0.107910,0.147728,0.186884,0.254450,0.259428,0.249010
salesVolume_diff_1,0.107910,1.000000,0.557657,0.394564,0.394201,0.432899,0.360183
salesVolume_diff_2,0.147728,0.557657,1.000000,0.633690,0.511264,0.535180,0.525734
salesVolume_diff_3,0.186884,0.394564,0.633690,1.000000,0.654475,0.563027,0.559776
salesVolume_diff_4,0.254450,0.394201,0.511264,0.654475,1.000000,0.696295,0.595192
salesVolume_diff_5,0.259428,0.432899,0.535180,0.563027,0.696295,1.000000,0.719325
salesVolume_diff_6,0.249010,0.360183,0.525734,0.559776,0.595192,0.719325,1.000000


In [76]:
df[colst2].corr()

,salesVolume,salesVolume_diff_1_ratio,salesVolume_diff_2_ratio,salesVolume_diff_3_ratio,salesVolume_diff_4_ratio,salesVolume_diff_5_ratio,salesVolume_diff_6_ratio
salesVolume,1.000000,0.064131,0.072091,0.087435,0.098760,0.126508,0.126299
salesVolume_diff_1_ratio,0.064131,1.000000,0.626528,0.428799,0.444401,0.356146,0.349852
salesVolume_diff_2_ratio,0.072091,0.626528,1.000000,0.639393,0.483940,0.487094,0.507620
salesVolume_diff_3_ratio,0.087435,0.428799,0.639393,1.000000,0.561746,0.477688,0.545929
salesVolume_diff_4_ratio,0.098760,0.444401,0.483940,0.561746,1.000000,0.545312,0.502245
salesVolume_diff_5_ratio,0.126508,0.356146,0.487094,0.477688,0.545312,1.000000,0.662844
salesVolume_diff_6_ratio,0.126299,0.349852,0.507620,0.545929,0.502245,0.662844,1.000000


In [77]:
df[colst3].corr()

,salesVolume,popularity_diff_1,popularity_diff_2,popularity_diff_3,popularity_diff_4,popularity_diff_5,popularity_diff_6
salesVolume,1.000000,-0.250431,-0.232318,-0.218891,-0.212434,-0.218227,-0.224873
popularity_diff_1,-0.250431,1.000000,0.846195,0.712564,0.682501,0.658717,0.633052
popularity_diff_2,-0.232318,0.846195,1.000000,0.844958,0.710579,0.680544,0.653366
popularity_diff_3,-0.218891,0.712564,0.844958,1.000000,0.844732,0.709074,0.675979
popularity_diff_4,-0.212434,0.682501,0.710579,0.844732,1.000000,0.842168,0.702901
popularity_diff_5,-0.218227,0.658717,0.680544,0.709074,0.842168,1.000000,0.838946
popularity_diff_6,-0.224873,0.633052,0.653366,0.675979,0.702901,0.838946,1.000000


In [78]:
df[colst4].corr()

,salesVolume,popularity_diff_1_ratio,popularity_diff_2_ratio,popularity_diff_3_ratio,popularity_diff_4_ratio,popularity_diff_5_ratio,popularity_diff_6_ratio
salesVolume,1.000000,0.137684,0.138250,0.143867,0.149583,0.157427,0.160738
popularity_diff_1_ratio,0.137684,1.000000,0.708760,0.506080,0.482498,0.497465,0.496283
popularity_diff_2_ratio,0.138250,0.708760,1.000000,0.744597,0.523265,0.524129,0.498864
popularity_diff_3_ratio,0.143867,0.506080,0.744597,1.000000,0.733903,0.578961,0.539575
popularity_diff_4_ratio,0.149583,0.482498,0.523265,0.733903,1.000000,0.792505,0.620146
popularity_diff_5_ratio,0.157427,0.497465,0.524129,0.578961,0.792505,1.000000,0.786829
popularity_diff_6_ratio,0.160738,0.496283,0.498864,0.539575,0.620146,0.786829,1.000000


In [58]:
df[df.time_id==25]

,province,model,bodyType,regYear,regMonth,salesVolume,popularity,carCommentVolum,newsReplyVolum,id,time_id,salesVolume_last_1,salesVolume_last_2,salesVolume_last_3,salesVolume_last_4,salesVolume_last_5,salesVolume_last_6,salesVolume_sum_2,salesVolume_sum_3,salesVolume_sum_4,salesVolume_sum_5,salesVolume_diff_1_2,salesVolume_diff_1_2_ratio,salesVolume_diff_2_3,salesVolume_diff_2_3_ratio,salesVolume_diff_3_4,salesVolume_diff_3_4_ratio,salesVolume_diff_4_5,salesVolume_diff_4_5_ratio,salesVolume_diff_5_6,salesVolume_diff_5_6_ratio
31680,上海,3c974920a76ac9c1,SUV,2018,1,NaN,NaN,NaN,NaN,1.0,25,312.0,286.0,270.0,308.0,298.0,240.0,598.0,868.0,1176.0,1474.0,26.0,0.090592,16.0,0.059041,-38.0,-0.122977,10.0,0.033445,58.0,0.240664
31681,云南,3c974920a76ac9c1,SUV,2018,1,NaN,NaN,NaN,NaN,2.0,25,331.0,333.0,298.0,306.0,323.0,362.0,664.0,962.0,1268.0,1591.0,-2.0,-0.005988,35.0,0.117057,-8.0,-0.026059,-17.0,-0.052469,-39.0,-0.107438
31682,内蒙古,3c974920a76ac9c1,SUV,2018,1,NaN,NaN,NaN,NaN,3.0,25,173.0,180.0,134.0,176.0,170.0,199.0,353.0,487.0,663.0,833.0,-7.0,-0.038674,46.0,0.340741,-42.0,-0.237288,6.0,0.035088,-29.0,-0.145000
31683,北京,3c974920a76ac9c1,SUV,2018,1,NaN,NaN,NaN,NaN,4.0,25,275.0,297.0,226.0,293.0,310.0,302.0,572.0,798.0,1091.0,1401.0,-22.0,-0.073826,71.0,0.312775,-67.0,-0.227891,-17.0,-0.054662,8.0,0.026403
31684,四川,3c974920a76ac9c1,SUV,2018,1,NaN,NaN,NaN,NaN,5.0,25,456.0,332.0,303.0,449.0,330.0,304.0,788.0,1091.0,1540.0,1870.0,124.0,0.372372,29.0,0.095395,-146.0,-0.324444,119.0,0.359517,26.0,0.085246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32995,福建,a9a43d1a7ecbe75d,SUV,2018,1,NaN,NaN,NaN,NaN,1338.0,25,112.0,130.0,89.0,100.0,106.0,78.0,242.0,331.0,431.0,537.0,-18.0,-0.137405,41.0,0.455556,-11.0,-0.108911,-6.0,-0.056075,28.0,0.354430
32996,辽宁,a9a43d1a7ecbe75d,SUV,2018,1,NaN,NaN,NaN,NaN,1339.0,25,104.0,119.0,99.0,123.0,140.0,79.0,223.0,322.0,445.0,585.0,-15.0,-0.125000,20.0,0.200000,-24.0,-0.193548,-17.0,-0.120567,61.0,0.762500
32997,重庆,a9a43d1a7ecbe75d,SUV,2018,1,NaN,NaN,NaN,NaN,1340.0,25,125.0,123.0,92.0,117.0,136.0,151.0,248.0,340.0,457.0,593.0,2.0,0.016129,31.0,0.333333,-25.0,-0.211864,-19.0,-0.138686,-15.0,-0.098684
32998,陕西,a9a43d1a7ecbe75d,SUV,2018,1,NaN,NaN,NaN,NaN,1341.0,25,265.0,277.0,301.0,207.0,338.0,256.0,542.0,843.0,1050.0,1388.0,-12.0,-0.043165,-24.0,-0.079470,94.0,0.451923,-131.0,-0.386431,82.0,0.319066


In [59]:
df[df.time_id==26]

,province,model,bodyType,regYear,regMonth,salesVolume,popularity,carCommentVolum,newsReplyVolum,id,time_id,salesVolume_last_1,salesVolume_last_2,salesVolume_last_3,salesVolume_last_4,salesVolume_last_5,salesVolume_last_6,salesVolume_sum_2,salesVolume_sum_3,salesVolume_sum_4,salesVolume_sum_5,salesVolume_diff_1_2,salesVolume_diff_1_2_ratio,salesVolume_diff_2_3,salesVolume_diff_2_3_ratio,salesVolume_diff_3_4,salesVolume_diff_3_4_ratio,salesVolume_diff_4_5,salesVolume_diff_4_5_ratio,salesVolume_diff_5_6,salesVolume_diff_5_6_ratio
33000,上海,3c974920a76ac9c1,SUV,2018,2,NaN,NaN,NaN,NaN,1343.0,26,NaN,312.0,286.0,270.0,308.0,298.0,NaN,NaN,NaN,NaN,NaN,NaN,26.0,0.090592,16.0,0.059041,-38.0,-0.122977,10.0,0.033445
33001,云南,3c974920a76ac9c1,SUV,2018,2,NaN,NaN,NaN,NaN,1344.0,26,NaN,331.0,333.0,298.0,306.0,323.0,NaN,NaN,NaN,NaN,NaN,NaN,-2.0,-0.005988,35.0,0.117057,-8.0,-0.026059,-17.0,-0.052469
33002,内蒙古,3c974920a76ac9c1,SUV,2018,2,NaN,NaN,NaN,NaN,1345.0,26,NaN,173.0,180.0,134.0,176.0,170.0,NaN,NaN,NaN,NaN,NaN,NaN,-7.0,-0.038674,46.0,0.340741,-42.0,-0.237288,6.0,0.035088
33003,北京,3c974920a76ac9c1,SUV,2018,2,NaN,NaN,NaN,NaN,1346.0,26,NaN,275.0,297.0,226.0,293.0,310.0,NaN,NaN,NaN,NaN,NaN,NaN,-22.0,-0.073826,71.0,0.312775,-67.0,-0.227891,-17.0,-0.054662
33004,四川,3c974920a76ac9c1,SUV,2018,2,NaN,NaN,NaN,NaN,1347.0,26,NaN,456.0,332.0,303.0,449.0,330.0,NaN,NaN,NaN,NaN,NaN,NaN,124.0,0.372372,29.0,0.095395,-146.0,-0.324444,119.0,0.359517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34315,福建,a9a43d1a7ecbe75d,SUV,2018,2,NaN,NaN,NaN,NaN,2680.0,26,NaN,112.0,130.0,89.0,100.0,106.0,NaN,NaN,NaN,NaN,NaN,NaN,-18.0,-0.137405,41.0,0.455556,-11.0,-0.108911,-6.0,-0.056075
34316,辽宁,a9a43d1a7ecbe75d,SUV,2018,2,NaN,NaN,NaN,NaN,2681.0,26,NaN,104.0,119.0,99.0,123.0,140.0,NaN,NaN,NaN,NaN,NaN,NaN,-15.0,-0.125000,20.0,0.200000,-24.0,-0.193548,-17.0,-0.120567
34317,重庆,a9a43d1a7ecbe75d,SUV,2018,2,NaN,NaN,NaN,NaN,2682.0,26,NaN,125.0,123.0,92.0,117.0,136.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.016129,31.0,0.333333,-25.0,-0.211864,-19.0,-0.138686
34318,陕西,a9a43d1a7ecbe75d,SUV,2018,2,NaN,NaN,NaN,NaN,2683.0,26,NaN,265.0,277.0,301.0,207.0,338.0,NaN,NaN,NaN,NaN,NaN,NaN,-12.0,-0.043165,-24.0,-0.079470,94.0,0.451923,-131.0,-0.386431


In [48]:
df[df.time_id==7]

,province,model,bodyType,regYear,regMonth,salesVolume,popularity,carCommentVolum,newsReplyVolum,time_id,salesVolume_last_1,salesVolume_last_2,salesVolume_last_3,salesVolume_last_4,salesVolume_last_5,salesVolume_last_6,salesVolume_sum_2,salesVolume_sum_3,salesVolume_sum_4,salesVolume_sum_5,salesVolume_diff_1_2,salesVolume_diff_1_2_ratio,salesVolume_diff_2_3,salesVolume_diff_2_3_ratio,salesVolume_diff_3_4,salesVolume_diff_3_4_ratio,salesVolume_diff_4_5,salesVolume_diff_4_5_ratio,salesVolume_diff_5_6,salesVolume_diff_5_6_ratio
7920,上海,3c974920a76ac9c1,SUV,2016,7,260,1171,37,13106,7,235.0,271.0,266.0,323.0,134.0,292.0,506.0,772.0,1095.0,1229.0,-36.0,-0.132353,5.0,0.018727,-57.0,-0.175926,189.0,1.400000,-158.0,-0.539249
7921,云南,3c974920a76ac9c1,SUV,2016,7,419,1316,37,13106,7,279.0,308.0,216.0,260.0,227.0,466.0,587.0,803.0,1063.0,1290.0,-29.0,-0.093851,92.0,0.423963,-44.0,-0.168582,33.0,0.144737,-239.0,-0.511777
7922,内蒙古,3c974920a76ac9c1,SUV,2016,7,187,1103,37,13106,7,164.0,236.0,149.0,206.0,123.0,257.0,400.0,549.0,755.0,878.0,-72.0,-0.303797,87.0,0.580000,-57.0,-0.275362,83.0,0.669355,-134.0,-0.519380
7923,北京,3c974920a76ac9c1,SUV,2016,7,322,2450,37,13106,7,351.0,445.0,404.0,362.0,160.0,408.0,796.0,1200.0,1562.0,1722.0,-94.0,-0.210762,41.0,0.101235,42.0,0.115702,202.0,1.254658,-248.0,-0.606357
7924,四川,3c974920a76ac9c1,SUV,2016,7,335,3165,37,13106,7,329.0,360.0,373.0,358.0,311.0,610.0,689.0,1062.0,1420.0,1731.0,-31.0,-0.085873,-13.0,-0.034759,15.0,0.041783,47.0,0.150641,-299.0,-0.489362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,福建,a9a43d1a7ecbe75d,SUV,2016,7,92,241,329,1849,7,64.0,99.0,69.0,88.0,45.0,131.0,163.0,232.0,320.0,365.0,-35.0,-0.350000,30.0,0.428571,-19.0,-0.213483,43.0,0.934783,-86.0,-0.651515
9236,辽宁,a9a43d1a7ecbe75d,SUV,2016,7,136,323,329,1849,7,116.0,133.0,97.0,120.0,60.0,141.0,249.0,346.0,466.0,526.0,-17.0,-0.126866,36.0,0.367347,-23.0,-0.190083,60.0,0.983607,-81.0,-0.570423
9237,重庆,a9a43d1a7ecbe75d,SUV,2016,7,146,214,329,1849,7,304.0,135.0,159.0,110.0,103.0,178.0,439.0,598.0,708.0,811.0,169.0,1.242647,-24.0,-0.150000,49.0,0.441441,7.0,0.067308,-75.0,-0.418994
9238,陕西,a9a43d1a7ecbe75d,SUV,2016,7,273,453,329,1849,7,217.0,260.0,190.0,179.0,88.0,221.0,477.0,667.0,846.0,934.0,-43.0,-0.164751,70.0,0.366492,11.0,0.061111,91.0,1.022472,-133.0,-0.599099


In [21]:
df.to_pickle('traindf.pkl')

In [22]:
df.shape

(36960, 31)

In [23]:
df.dropna().shape

(0, 31)